## Superpixel Generation

This notebook is to add a superpixel map for each image label (`.npy` format).

In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import glob

import matplotlib.pyplot as plt 

from misc.utils import rm_n_mkdir

In [2]:
img_ext = '.npy'
step_size = [256, 256] # should match self.train_mask_shape (config.py) 
win_size  = [512, 512] # should be at least twice time larger than 
                       # self.train_base_shape (config.py) to reduce 
                       # the padding effect during augmentation
region_size = 32

img_dir = "/home1/gzy/BreastCancerSeg/DATA/Train/%dx%d_%dx%d" % \
                    (win_size[0], win_size[1], step_size[0], step_size[1])
save_dir = "/home1/gzy/BreastCancerSeg/DATA/Train/%dx%d_%dx%d_slic%d" % \
                    (win_size[0], win_size[1], step_size[0], step_size[1], region_size)

####

file_list = glob.glob('%s/*%s' % (img_dir, img_ext))
file_list.sort()

rm_n_mkdir(save_dir)

In [8]:
for i in tqdm(range(len(file_list))):
    ori_path = file_list[i]
    basename = os.path.basename(ori_path)

    ori = np.load(ori_path)
    img = ori[...,:3].copy()
    img = img.astype(np.uint8)

    # super-pixel
    slic = cv2.ximgproc.createSuperpixelSLIC(img, region_size=region_size)
    slic.iterate(10)     #
    mask_slic = slic.getLabelContourMask() #
    label_slic = slic.getLabels()        #
    
    label_slic = np.expand_dims(label_slic, axis=-1)
    label_slic = label_slic.astype('int32')

    np.save(os.path.join(save_dir, basename), label_slic)

100%|██████████| 1312/1312 [03:10<00:00,  7.81it/s]
